In [2]:
%pip install pandas backtesting

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 3.4 MB/s eta 0:00:00a 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd

from backtesting import Backtest, Strategy
from backtesting.test import BTCUSD


In [ ]:
class HMMStrategy(Strategy):
    def init(self):
        # Load state column as a strategy indicator
        self.state = self.data.df['state'].values

    def next(self):
        current_state = self.state[self.i]
        
        # Example rule:
        # Buy when bullish, sell when bearish
        if current_state == 2:  # bullish
            if not self.position:
                self.buy()
        elif current_state == 0:  # bearish
            if self.position:
                self.position.close()

            Open  High   Low  Close        Volume
2012-01-31  4.58  7.38  3.80   5.55   2012.253436
2012-02-29  5.55  6.50  3.80   4.99   4761.609081
2012-03-31  4.99  5.44  4.54   4.92   9049.626282
2012-04-30  4.92  5.43  4.69   5.00  16484.167772
2012-05-31  5.00  5.15  4.86   5.14  18520.103801


In [ ]:
df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)

# Drop NaN or misaligned rows
df = df.dropna()

# Run backtest
bt = Backtest(df, HMMStrategy, cash=10000, commission=0.006)
stats = bt.run()
bt.plot()